# Fine-tuning AOT+ Model on Extracted Frames Dataset

This notebook provides a template for fine-tuning AOT+ (or related) models on the `EXTRACTED_FRAMES` dataset. The `EXTRACTED_FRAMES` dataset is designed to load images and their corresponding polygon annotations from JSON files, making it suitable for custom datasets where annotations are in this format.

**Key Steps:**
1. Configure parameters (experiment name, model, paths, hyperparameters).
2. Construct the training command.
3. Set environment variables (like `CUDA_VISIBLE_DEVICES`).
4. Execute the training script (`tools/train.py`).

**Before Running:**
- Ensure your `EXTRACTED_FRAMES` dataset is correctly placed (default assumption: `./extracted_frames/`).
- **Crucially, update the `pretrained_model_path` variable to point to your pretrained model weights.**
- Adjust other parameters like `model`, `batch_size`, `total_steps`, and GPU settings as needed.

In [ ]:
# Experiment Settings
exp_name = "finetune_extracted_notebook"
model = "r50_aotl"  # Options: "aott", "aots", "aotb", "aotl", "r50_deaotl", "swinb_aotl", etc.
stage = "default"    # Adjust if a specific fine-tuning stage or pretrained model stage is needed
dataset_name = "EXTRACTED_FRAMES"

# Path to your pretrained model (VERY IMPORTANT - UPDATE THIS PATH)
pretrained_model_path = ""  # e.g., "./pretrain_models/r50_aotl.pth" or "/path/to/your/model.pth"

# GPU Configuration
gpu_num = 1       # Number of GPUs to use
devices = "0"     # Comma-separated list of GPU IDs (e.g., "0" or "0,1")

# Training Hyperparameters
batch_size = 2    # Adjust based on your GPU memory
total_steps = 10000 # Total number of training steps
use_amp = True    # Enable Automatic Mixed Precision (if supported)
fix_random_seed = True # For reproducibility

# Optional: Log directory
log_directory = f"./logs_{exp_name}" # Example log directory

# Path to the training script
train_script_path = "tools/train.py"

# Basic check for pretrained_model_path (encourage user to fill it)
if not pretrained_model_path:
    print("WARNING: `pretrained_model_path` is not set. Fine-tuning usually requires a pretrained model.")
    print("Please update it to the path of your .pth model file.")

## 2. Constructing the Training Command

This cell assembles the command-line arguments for `tools/train.py` based on the parameters defined above.

In [ ]:
command = ["python", train_script_path]
command.extend(["--exp_name", exp_name])
command.extend(["--stage", stage])
command.extend(["--model", model])
command.extend(["--datasets", dataset_name])
command.extend(["--gpu_num", str(gpu_num)])
command.extend(["--batch_size", str(batch_size)])
command.extend(["--total_step", str(total_steps)])

if use_amp:
    command.append("--amp")
if fix_random_seed:
    command.append("--fix_random")

if pretrained_model_path:  # Only add if a path is provided
    command.extend(["--pretrained_path", pretrained_model_path])
else:
    print("INFO: Proceeding without a specified `pretrained_model_path`. The model might train from scratch or use default weights if defined in its config.")

# To use a custom log directory, uncomment the following line:
# command.extend(["--log", log_directory])

print("Constructed command:")
# Pretty print for readability
import shlex
print(shlex.join(command))

## 3. Setting CUDA_VISIBLE_DEVICES

To ensure the training script uses the specified GPUs, we set the `CUDA_VISIBLE_DEVICES` environment variable. This is particularly important in multi-GPU systems.

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = devices
print(f"CUDA_VISIBLE_DEVICES set to: {os.environ['CUDA_VISIBLE_DEVICES']}")

## 4. Executing the Training Command

This cell runs the training script using `subprocess.run`. This method is generally preferred over `!python` (shell command) in notebooks because it offers better control over the process, allows capturing output programmatically, and integrates more smoothly with Python variables.

**Note:** Training can take a significant amount of time depending on the dataset size, model complexity, number of steps, and hardware.

In [ ]:
import subprocess

print("\nStarting training... This may take a while.")
print(f"Executing command: {shlex.join(command)}")
print(f"Output will be streamed below if running in a typical Jupyter environment that handles subprocess output well.\n")

# Using subprocess.run for better control. 
# For live output in Jupyter, Popen might be better, but requires more complex handling.
# result = subprocess.run(command, capture_output=True, text=True, check=False)

# Simpler alternative for direct output to notebook (less control over capture):
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True)

if process.stdout:
    for line in iter(process.stdout.readline, ''):
        print(line, end='')
    process.stdout.close()

return_code = process.wait()

if return_code == 0:
    print("\n--- Training successfully completed. ---")
else:
    print(f"\n--- Training exited with code: {return_code} ---")

# If you used result = subprocess.run(command, capture_output=True, text=True):
# print("\n--- Training Output (stdout) ---")
# print(result.stdout)
# if result.stderr:
#     print("\n--- Training Errors (stderr) ---")
#     print(result.stderr)
# print("\nTraining finished.")

## 5. Important Notes

*   **Update `pretrained_model_path`**: This is the most critical parameter for fine-tuning. Ensure it points to a valid `.pth` model file.
*   **Adjust Parameters**: Modify `batch_size` according to your GPU memory. Change `total_steps` based on how long you want to fine-tune. Select the correct `model` architecture. Update `gpu_num` and `devices` for your specific hardware setup.
*   **Logs**: If you uncommented the `--log` argument in the command construction, training logs (including print statements, TensorBoard files, and saved checkpoints) will be saved to the specified directory (e.g., `./logs_finetune_extracted_notebook/`).
*   **Dataset Location**: The `ExtractedFramesTrain` dataset loader defaults to looking for data in `./extracted_frames/`. Ensure your images and JSON annotations are there.
*   **Kernel Interrupts**: If you interrupt the kernel during the `subprocess.run` execution, it might not immediately terminate the child process (`tools/train.py`). You might need to manually stop it if it continues running in the background.